# This examples shows who to download files from the ONC server

In [1]:
import strawb

In [2]:
onc_downloader = strawb.ONCDownloader(showInfo=False)

filters = {'deviceCode': 'TUMMUONTRACKER001',
           'dateFrom': '2021-05-03T00:00:00.000Z',
           'dateTo': '2021-05-03T01:59:59.000Z',
           'extension': 'hdf5'}
# another filer paramter is: 'dataProductCode': 'SMRD'

# Show available files on the ONC server

In [3]:
onc_downloader.getListByDevice(filters=filters, allPages=True)

{'next': None,
 'queryUrl': 'https://data.oceannetworks.ca/api/archivefiles?deviceCode=TUMMUONTRACKER001&dateFrom=2021-05-03T00%3A00%3A00.000Z&dateTo=2021-05-03T01%3A59%3A59.000Z&token=0db751f8-9430-47af-bc11-ed6691b38e22&method=getListByDevice',
 'files': ['TUMMUONTRACKER001_20210503T000000.000Z-SDAQ-MUON.hdf5',
  'TUMMUONTRACKER001_20210503T000000.000Z-SDAQ-MODULE.hdf5',
  'TUMMUONTRACKER001_20210503T000518.304Z-SDAQ-MUON.hdf5',
  'TUMMUONTRACKER001_20210503T001131.154Z-SDAQ-MUON.hdf5',
  'TUMMUONTRACKER001_20210503T001715.116Z-SDAQ-MODULE.hdf5',
  'TUMMUONTRACKER001_20210503T002734.556Z-SDAQ-MUON.hdf5',
  'TUMMUONTRACKER001_20210503T011715.024Z-SDAQ-MODULE.hdf5',
  'TUMMUONTRACKER001_20210503T012744.797Z-SDAQ-MUON.hdf5']}

## Get the same as Pandas DataFrame + structured 'outPath'

In [4]:
df_result = onc_downloader.download_structured(dev_codes=[filters['deviceCode']],
                                               date_from=filters['dateFrom'],
                                               date_to=filters['dateTo'],
                                               download=False)

df_result  # show the pandas DataFrame

In total: 13 files; exclude: 0; 


,archiveLocation,archivedDate,compression,dataProductCode,dateFrom,dateTo,deviceCode,fileSize,filename,modifyDate,path,uncompressedFileSize,outPath,synced,fullPath
0,/NA_Archive_04,2021-05-03 01:36:58.550000+00:00,None,MTRD,2021-05-02 23:27:24.668000+00:00,2021-05-03 00:00:00.996000+00:00,TUMMUONTRACKER001,16573352,TUMMUONTRACKER001_20210502T232724.668Z-MUON.hld,2021-05-03 01:36:59.041000+00:00,02/83/34,16573352,/Users/kilian/PycharmProjects/strawb_package/r...,True,/Users/kilian/PycharmProjects/strawb_package/r...
1,/NA_Archive_04,2021-05-03 01:36:59.107000+00:00,None,MTSD,2021-05-03 00:00:00+00:00,2021-05-03 00:05:01.444000+00:00,TUMMUONTRACKER001,152952,TUMMUONTRACKER001_20210503T000000.000Z-SDAQ-MU...,2021-05-03 01:36:59.149000+00:00,02/83/34,152952,/Users/kilian/PycharmProjects/strawb_package/r...,True,/Users/kilian/PycharmProjects/strawb_package/r...
2,/NA_Archive_04,2021-05-05 01:37:21.600000+00:00,None,SMRD,2021-05-03 00:00:00+00:00,2021-05-03 23:59:59.999000+00:00,TUMMUONTRACKER001,103792,TUMMUONTRACKER001_20210503T000000.000Z-SDAQ-MO...,2021-05-05 01:37:21.652000+00:00,02/96/41,103792,/Users/kilian/PycharmProjects/strawb_package/r...,True,/Users/kilian/PycharmProjects/strawb_package/r...
3,/NA_Archive_04,2021-05-04 05:09:41.807000+00:00,gz,LF,2021-05-03 00:00:00+00:00,2021-05-04 00:00:00+00:00,TUMMUONTRACKER001,379354,TUMMUONTRACKER001_20210503T000000.000Z.txt,2021-05-04 05:09:42.108000+00:00,02/91/08,3534086,/Users/kilian/PycharmProjects/strawb_package/r...,True,/Users/kilian/PycharmProjects/strawb_package/r...
4,/NA_Archive_04,2021-05-03 01:36:59.222000+00:00,None,MTRD,2021-05-03 00:00:01.223000+00:00,2021-05-03 00:05:01.105000+00:00,TUMMUONTRACKER001,472280,TUMMUONTRACKER001_20210503T000001.223Z-MUON.hld,2021-05-03 01:36:59.272000+00:00,02/83/34,472280,/Users/kilian/PycharmProjects/strawb_package/r...,True,/Users/kilian/PycharmProjects/strawb_package/r...
5,/NA_Archive_04,2021-05-03 01:36:59.353000+00:00,None,MTSD,2021-05-03 00:05:18.304000+00:00,2021-05-03 00:11:28.615000+00:00,TUMMUONTRACKER001,190368,TUMMUONTRACKER001_20210503T000518.304Z-SDAQ-MU...,2021-05-03 01:36:59.395000+00:00,02/83/34,190368,/Users/kilian/PycharmProjects/strawb_package/r...,True,/Users/kilian/PycharmProjects/strawb_package/r...
6,/NA_Archive_04,2021-05-05 01:37:21.807000+00:00,None,MTSD,2021-05-03 00:11:31.154000+00:00,2021-05-03 23:59:59.999000+00:00,TUMMUONTRACKER001,183632,TUMMUONTRACKER001_20210503T001131.154Z-SDAQ-MU...,2021-05-05 01:37:21.860000+00:00,02/96/41,183632,/Users/kilian/PycharmProjects/strawb_package/r...,True,/Users/kilian/PycharmProjects/strawb_package/r...
7,/NA_Archive_04,2021-05-05 01:37:22.063000+00:00,None,SMRD,2021-05-03 00:17:15.116000+00:00,2021-05-03 23:59:59.999000+00:00,TUMMUONTRACKER001,186480,TUMMUONTRACKER001_20210503T001715.116Z-SDAQ-MO...,2021-05-05 01:37:22.119000+00:00,02/96/41,186480,/Users/kilian/PycharmProjects/strawb_package/r...,True,/Users/kilian/PycharmProjects/strawb_package/r...
8,/NA_Archive_04,2021-05-05 01:37:22.249000+00:00,None,MTSD,2021-05-03 00:27:34.556000+00:00,2021-05-03 23:59:59.999000+00:00,TUMMUONTRACKER001,346272,TUMMUONTRACKER001_20210503T002734.556Z-SDAQ-MU...,2021-05-05 01:37:22.315000+00:00,02/96/41,346272,/Users/kilian/PycharmProjects/strawb_package/r...,True,/Users/kilian/PycharmProjects/strawb_package/r...
9,/NA_Archive_04,2021-05-03 01:36:59.498000+00:00,None,MTRD,2021-05-03 00:27:34.557000+00:00,2021-05-03 01:00:00.073000+00:00,TUMMUONTRACKER001,16294792,TUMMUONTRACKER001_20210503T002734.557Z-MUON.hld,2021-05-03 01:37:01.233000+00:00,02/83/34,16294792,/Users/kilian/PycharmProjects/strawb_package/r...,True,/Users/kilian/PycharmProjects/strawb_package/r...


## and download the files listed in the DataFrame which are <10MB

In [5]:
onc_downloader.getDirectFiles(filters_or_result=df_result[df_result['fileSize']<10e6])  # <10MB

Skipping 10 files.
Downloaded - Directory: /Users/kilian/PycharmProjects/strawb_package/raw_module_data; Files: 0; Size: 0 Bytes; Time: 0.000 seconds; Speed: 0 Bytes/s


# Access the info about the download

In [6]:
onc_downloader.result

{'downloadResults': [],
 'stats': {'totalSize': 0, 'downloadTime': 0, 'fileCount': 0}}

# Get possible dataProductCodes and dataProductName for a device

In [7]:
# get posible dataProductCodes and dataProductName for the device
onc_downloader.getDataProducts({'deviceCode': filters['deviceCode']})

[{'dataProductCode': 'LF',
  'dataProductName': 'Log File',
  'extension': 'txt',
  'hasDeviceData': True,
  'hasPropertyData': False,
  'helpDocument': 'https://wiki.oceannetworks.ca/display/DP/4'},
 {'dataProductCode': 'MSQAQCR',
  'dataProductName': 'Manual Scalar QAQC Results',
  'extension': 'qaqc',
  'hasDeviceData': True,
  'hasPropertyData': False,
  'helpDocument': 'https://wiki.oceannetworks.ca/display/DP/106'},
 {'dataProductCode': 'MTRD',
  'dataProductName': 'Muon Tracker Raw Data',
  'extension': 'hld',
  'hasDeviceData': True,
  'hasPropertyData': False,
  'helpDocument': 'https://wiki.oceannetworks.ca/display/DP/158'},
 {'dataProductCode': 'MTSD',
  'dataProductName': 'Muon Tracker Sensor Data',
  'extension': 'hdf5',
  'hasDeviceData': True,
  'hasPropertyData': False,
  'helpDocument': 'https://wiki.oceannetworks.ca/display/DP/168'},
 {'dataProductCode': 'MTTOT',
  'dataProductName': 'Muon Tracker Time Over Threshold Data',
  'extension': 'txt',
  'hasDeviceData': Tru

In [8]:
# get posible dataProductCodes and dataProductName for the device only for hdf5-files
onc_downloader.getDataProducts({'deviceCode': filters['deviceCode'], 'extension': 'hdf5'})

[{'dataProductCode': 'MTSD',
  'dataProductName': 'Muon Tracker Sensor Data',
  'extension': 'hdf5',
  'hasDeviceData': True,
  'hasPropertyData': False,
  'helpDocument': 'https://wiki.oceannetworks.ca/display/DP/168'},
 {'dataProductCode': 'SMRD',
  'dataProductName': 'Standard Module Raw Data',
  'extension': 'hdf5',
  'hasDeviceData': True,
  'hasPropertyData': False,
  'helpDocument': 'https://wiki.oceannetworks.ca/display/DP/157'}]